In [1]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

c:\Users\WONJONGHYEON\anaconda3\envs\CP2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [196]:
CFG = {
    'EPOCHS':5,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':256,
    'SEED':41
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
!unzip open.zip

Archive:  open.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [104]:
df_kind=pd.read_csv(r'C:\Users\WONJONGHYEON\OneDrive\바탕 화면\CP2\augment_csv\df_kind_aug.csv')
df_copy=df.copy()

In [156]:
df_kind['유형'].value_counts(dropna=False)

사실형    6980
대화형    4032
예측형    4012
추론형    3700
Name: 유형, dtype: int64

In [164]:
test=pd.read_csv('test.csv')
test_copy=test.copy()

In [165]:
test

,ID,문장
0,TEST_0000,"장욱진의 ＇가족＇은 허물 없는 가족애를, 처음 공개되는 정약용의 ＇정효자전＇과 ＇정..."
1,TEST_0001,"조지 W 부시, 버락 오바마 전 대통령도 전쟁 위험 때문에 버린 카드다."
2,TEST_0002,지난해 1분기 128억원이었던 영업이익이 올해 1분기 505억원으로 급증했다.
3,TEST_0003,수상 작가와 맺으려던 계약서 내용 가운데 일부가 ＇독소 조항＇으로 해석돼 수정을 요...
4,TEST_0004,결국 최근 KDB산업은행은 대규모 손실 위기에 닥친 에어부산에 140억원 금융지원을...
...,...,...
7085,TEST_7085,"2020 세계국가편람 모바일 앱은 세계 216개국의 국가개황과 주요 경제지표, 사회..."
7086,TEST_7086,탈세계화 징후들이 반갑지 않은 이유다.
7087,TEST_7087,"틱톡은 6월 ＇인터넷 안전의 달＇을 맞아 올바른 개인정보 보호 관리 방법, 앱 내 ..."
7088,TEST_7088,만약 3개월 간 채굴자들의 투표를 거쳐 2/3 이상의 해시파워가 ＇채굴세＇ 도입에 ...


In [160]:
train, val= train_test_split(df_kind, test_size=0.2, random_state=CFG['SEED'], stratify=df_kind['유형'])

In [163]:
train

,문장,유형
8858,제가 항상 가슴에 새겼던 것이 개인적인 말은 창의적인 것이었습니다,대화형
6455,그만큼 주민들의 재건축 의지가 높다는 뜻으로 해석할 수 있다,추론형
1769,이에 a씨에게는 징역 1월에 집행유예 1년 삼성디스플레이 직원들에게는 징역 11월에...,사실형
1228,대표팀이 1월 1일과 1일 최종예선에서 만날 두 팀은 베네수엘라와 리투아니아다,사실형
3530,정교한 그래픽을 통해 애니메이션의 한 장면을 보는 듯한 시각미를 선사한다,사실형
...,...,...
14053,이날 코픽스 금리 상승으로 16일부터 뛰어오를 대출금리가 시중은행 것으로 전망된다,예측형
14841,올해 맞는 MBN건강미박람회는 브라보 앵콜 라이프를 위한 시니어를 삼고 다양한 정보...,예측형
7400,팬덤이 계속 유지가 될 수 있도록 단순한 등기 권리증 역할을 넘어 a 소유자들을 위...,추론형
6514,지난해 설비투자액이 1조1억원이었다는 데 비춰보면 1조1억원의 투자 증가 효과가 나...,추론형


In [162]:
val

,문장,유형
1145,추사 김정희 11는 완당집에서 우리나라에는 난초를 그리는 사람이 없었다 그런데 엎드...,사실형
11222,시짱 자치구 고원의 신장 자치구 인도 카슈미르 걸친 칭하이성 평균 해발고도는 1a입니다,대화형
17776,법무부는 글로벌 스탠더드에 맞는 법무행정 현대화 및 법제 정비 국제 형사사법 협력 ...,사실형
7343,미세먼지 농도는 전 권역이 좋음보통 수준을 보이겠으며 아침까지 중부 내륙과 남부지방...,추론형
3913,기존의 콘텐츠들이 삭제되고 전달력과 몰입감을 높여 이용자들이 재미를 느낄 수 있도록...,사실형
...,...,...
17486,숙면 특허를 개발을 위해 몽가타는 모터 드라이버 펌웨어에 대한 1건에 달하는 침대 ...,사실형
15807,비누가 있으면 있는대로 생각날 없는대로 없으면 씻어주는 게 좋다,추론형
13465,해상에서는 서해상을 중심으로 천둥번개가 함께 돌풍과 치는 곳이 있겠다,예측형
14214,아울러 채굴 난이도 상승을 고려할 채굴 수입에서 커질 비용이 차지하는 비중이 점점 ...,예측형


In [166]:
vectorizer = TfidfVectorizer(min_df = 4, analyzer = 'word', ngram_range=(1, 2))
vectorizer.fit(np.array(df_kind["문장"]))

train_vec = vectorizer.transform(train["문장"])
val_vec = vectorizer.transform(val["문장"])
test_vec = vectorizer.transform(test["문장"])

print(train_vec.shape, val_vec.shape, test_vec.shape)

(14979, 25270) (3745, 25270) (7090, 25270)


In [167]:
# 2. Label Encoding (유형, 극성, 시제, 확실성)
type_le = preprocessing.LabelEncoder()
train["유형"] = type_le.fit_transform(train["유형"].values)
val["유형"] = type_le.transform(val["유형"].values)


In [168]:
train

,문장,유형
8858,제가 항상 가슴에 새겼던 것이 개인적인 말은 창의적인 것이었습니다,0
6455,그만큼 주민들의 재건축 의지가 높다는 뜻으로 해석할 수 있다,3
1769,이에 a씨에게는 징역 1월에 집행유예 1년 삼성디스플레이 직원들에게는 징역 11월에...,1
1228,대표팀이 1월 1일과 1일 최종예선에서 만날 두 팀은 베네수엘라와 리투아니아다,1
3530,정교한 그래픽을 통해 애니메이션의 한 장면을 보는 듯한 시각미를 선사한다,1
...,...,...
14053,이날 코픽스 금리 상승으로 16일부터 뛰어오를 대출금리가 시중은행 것으로 전망된다,2
14841,올해 맞는 MBN건강미박람회는 브라보 앵콜 라이프를 위한 시니어를 삼고 다양한 정보...,2
7400,팬덤이 계속 유지가 될 수 있도록 단순한 등기 권리증 역할을 넘어 a 소유자들을 위...,3
6514,지난해 설비투자액이 1조1억원이었다는 데 비춰보면 1조1억원의 투자 증가 효과가 나...,3


In [ ]:
polarity_le = preprocessing.LabelEncoder()
train["극성"] = polarity_le.fit_transform(train["극성"].values)
val["극성"] = polarity_le.transform(val["극성"].values)

tense_le = preprocessing.LabelEncoder()
train["시제"] = tense_le.fit_transform(train["시제"].values)
val["시제"] = tense_le.transform(val["시제"].values)

certainty_le = preprocessing.LabelEncoder()
train["확실성"] = certainty_le.fit_transform(train["확실성"].values)
val["확실성"] = certainty_le.transform(val["확실성"].values)

In [170]:
train_type = train["유형"].values # sentence type
# train_polarity = train["극성"].values # sentence polarity
# train_tense = train["시제"].values # sentence tense
# train_certainty = train["확실성"].values # sentence certainty

train_labels = {
    'type' : train_type
    # 'polarity' : train_polarity,
    # 'tense' : train_tense,
    # 'certainty' : train_certainty
}

In [171]:
val_type = val["유형"].values # sentence type
#val_polarity = val["극성"].values # sentence polarity
#val_tense = val["시제"].values # sentence tense
#val_certainty = val["확실성"].values # sentence certainty

val_labels = {
    'type' : val_type,
#    'polarity' : val_polarity,
#    'tense' : val_tense,
#   'certainty' : val_certainty
}

In [172]:
class CustomDataset(Dataset):
    def __init__(self, st_vec, st_labels):
        self.st_vec = st_vec
        self.st_labels = st_labels

    def __getitem__(self, index):
        st_vector = torch.FloatTensor(self.st_vec[index].toarray()).squeeze(0)
        if self.st_labels is not None:
            st_type = self.st_labels['type'][index]
            #st_polarity = self.st_labels['polarity'][index]
            #st_tense = self.st_labels['tense'][index]
            #st_certainty = self.st_labels['certainty'][index]
            return st_vector, st_type#, st_polarity, st_tense, st_certainty
        else:
            return st_vector

    def __len__(self):
        return len(self.st_vec.toarray())

In [173]:
train_dataset = CustomDataset(train_vec, train_labels)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_vec, val_labels)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [179]:
train_dataset[741]

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), 2)

In [180]:
len(val_loader)

15

In [185]:
class BaseModel(nn.Module):
    def __init__(self, input_dim=25270):
        super(BaseModel, self).__init__()
        self.feature_extract = nn.Sequential(
            nn.Linear(in_features=input_dim, out_features=1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024, out_features=1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024, out_features=512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
        )
        self.type_classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features=512, out_features=4)
         )
        # self.polarity_classifier = nn.Sequential(
        #     nn.Dropout(p=0.3),
        #     nn.Linear(in_features=512, out_features=3),
        # )
        # self.tense_classifier = nn.Sequential(
        #     nn.Dropout(p=0.3),
        #     nn.Linear(in_features=512, out_features=3),
        # )
        # self.certainty_classifier = nn.Sequential(
        #     nn.Dropout(p=0.3),
        #     nn.Linear(in_features=512, out_features=2),
        # )
            
    def forward(self, x):
        x = self.feature_extract(x)
        # 문장 유형, 극성, 시제, 확실성을 각각 분류
        
        type_output = self.type_classifier(x)

        # polarity_output = self.polarity_classifier(x)

        # tense_output = self.tense_classifier(x)

        # certainty_output = self.certainty_classifier(x)

        return type_output#, polarity_output, tense_output, certainty_output

In [198]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    
    criterion = {
        'type' : nn.CrossEntropyLoss().to(device)
        # 'polarity' : nn.CrossEntropyLoss().to(device),
        # 'tense' : nn.CrossEntropyLoss().to(device),
        # 'certainty' : nn.CrossEntropyLoss().to(device)
    }
    
    best_loss = 999999
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for sentence, type_label in tqdm(iter(train_loader)): #, polarity_label, tense_label, certainty_label 
            sentence = sentence.to(device)
            type_label = type_label.long().to(device)

            # polarity_label = polarity_label.to(device)
            # tense_label = tense_label.to(device)
            # certainty_label = certainty_label.to(device)
            
            optimizer.zero_grad()
            
            type_logit = model(sentence) #, tense_logit, certainty_logit, polarity_logit  
            
            loss = criterion['type'](type_logit, type_label) # criterion['polarity'](polarity_logit, polarity_label) criterion['tense'](tense_logit, tense_label) criterion['certainty'](certainty_logit, certainty_label)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')
        
        if scheduler is not None:
            scheduler.step(val_loss)
            
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            
    return best_model

In [199]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    type_preds, polarity_preds, tense_preds, certainty_preds = [], [], [], []
    type_labels, polarity_labels, tense_labels, certainty_labels = [], [], [], []
    
    
    with torch.no_grad():
        for sentence, type_label in tqdm(iter(val_loader)): #, polarity_label, tense_label, certainty_label 
            sentence = sentence.to(device)
            type_label = type_label.long().to(device)
            # polarity_label = polarity_label.to(device)
            # tense_label = tense_label.to(device)
            # certainty_label = certainty_label.to(device)
            
            type_logit = model(sentence) #, polarity_logit, tense_logit, certainty_logit 
            
            loss = criterion['type'](type_logit, type_label) #criterion['polarity'](polarity_logit, polarity_label) criterion['tense'](tense_logit, tense_label) criterion['certainty'](certainty_logit, certainty_label)
            
            val_loss.append(loss.item())
            
            type_preds += type_logit.argmax(1).detach().cpu().numpy().tolist()
            type_labels += type_label.detach().cpu().numpy().tolist()
            
            # polarity_preds += polarity_logit.argmax(1).detach().cpu().numpy().tolist()
            # polarity_labels += polarity_label.detach().cpu().numpy().tolist()
            
            # tense_preds += tense_logit.argmax(1).detach().cpu().numpy().tolist()
            # tense_labels += tense_label.detach().cpu().numpy().tolist()
            
            # certainty_preds += certainty_logit.argmax(1).detach().cpu().numpy().tolist()
            # certainty_labels += certainty_label.detach().cpu().numpy().tolist()
    
    type_f1 = f1_score(type_labels, type_preds, average='weighted')
    # polarity_f1 = f1_score(polarity_labels, polarity_preds, average='weighted')
    # tense_f1 = f1_score(tense_labels, tense_preds, average='weighted')
    # certainty_f1 = f1_score(certainty_labels, certainty_preds, average='weighted')
    
    return np.mean(val_loss)#, type_f1, polarity_f1, tense_f1, certainty_f1

In [200]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|██████████| 15/15 [00:07<00:00,  2.09it/s]


Epoch : [1] Train Loss : [0.60287] Val Loss : [1.37453]


100%|██████████| 15/15 [00:04<00:00,  3.25it/s]


Epoch : [2] Train Loss : [0.07499] Val Loss : [0.25496]


100%|██████████| 15/15 [00:10<00:00,  1.49it/s]


Epoch : [3] Train Loss : [0.02561] Val Loss : [0.28336]


100%|██████████| 15/15 [00:07<00:00,  2.01it/s]


Epoch : [4] Train Loss : [0.01709] Val Loss : [0.29900]


100%|██████████| 15/15 [00:04<00:00,  3.04it/s]

Epoch : [5] Train Loss : [0.01373] Val Loss : [0.30923]
Epoch 00005: reducing learning rate of group 0 to 5.0000e-05.


In [83]:
test_dataset = CustomDataset(test_vec, None)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [84]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    type_preds, polarity_preds, tense_preds, certainty_preds = [], [], [], []
    
    with torch.no_grad():
        for sentence in tqdm(test_loader):
            sentence = sentence.to(device)
            
            type_logit, polarity_logit, tense_logit, certainty_logit = model(sentence)
            
            type_preds += type_logit.argmax(1).detach().cpu().numpy().tolist()
            # polarity_preds += polarity_logit.argmax(1).detach().cpu().numpy().tolist()
            # tense_preds += tense_logit.argmax(1).detach().cpu().numpy().tolist()
            # certainty_preds += certainty_logit.argmax(1).detach().cpu().numpy().tolist()
            
    return type_preds, polarity_preds, tense_preds, certainty_preds

In [85]:
type_preds= inference(model, test_loader, device) #, polarity_preds, tense_preds, certainty_preds 

  0%|          | 0/28 [00:00<?, ?it/s]

In [86]:
type_preds = type_le.inverse_transform(type_preds)
# polarity_preds = polarity_le.inverse_transform(polarity_preds)
# tense_preds = tense_le.inverse_transform(tense_preds)
# certainty_preds = certainty_le.inverse_transform(certainty_preds)

In [87]:
predictions = []
for type_pred, polarity_pred, tense_pred, certainty_pred in zip(type_preds, polarity_preds, tense_preds, certainty_preds):
    predictions.append(type_pred+'-'+polarity_pred+'-'+tense_pred+'-'+certainty_pred)

In [88]:
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = predictions

In [90]:
submit

,ID,label
0,TEST_0000,사실형-긍정-현재-확실
1,TEST_0001,사실형-긍정-현재-확실
2,TEST_0002,사실형-긍정-과거-확실
3,TEST_0003,사실형-긍정-과거-확실
4,TEST_0004,사실형-긍정-과거-확실
...,...,...
7085,TEST_7085,사실형-긍정-현재-확실
7086,TEST_7086,추론형-긍정-현재-확실
7087,TEST_7087,사실형-긍정-현재-확실
7088,TEST_7088,사실형-긍정-미래-확실


In [92]:
submit.to_csv('baseline_submit.csv', index=False, encoding="utf-8-sig")